In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    pass

Mounted at /content/drive


In [ ]:
!pip install pyheif

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 11.2 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import h5py
import cv2
from PIL import Image, ImageEnhance
from skimage import io
from skimage.transform import rotate
from io import BytesIO
import pyheif

In [ ]:
# Path to your HDF5 file
hdf5_file_path = '/content/drive/MyDrive/IS4242/processed_data/images_h5'

# Open the HDF5 file
with h5py.File(hdf5_file_path, 'r') as hf:
    # Access the dataset containing images
    images = hf['images'][:]

# Now 'images' variable contains your array of images
# You can use it as needed
print(f"Loaded HDF5 file with images of shape: {images.shape}")

Loaded HDF5 file with images of shape: (857, 128, 128, 3)


In [ ]:
# Function to resize and convert image to JPG format
def preprocess_image(image):
    # Resize image to a standard size (e.g., 128x128)
    resized_image = cv2.resize(image, (128, 128))

    # Convert image from HEIC to JPG (example for HEIC format)
    img_bytes = cv2.imencode('.jpg', resized_image)[1].tostring()
    pil_image = Image.open(BytesIO(img_bytes))
    jpg_image = np.array(pil_image)

    return jpg_image

# Function to apply image augmentation (brightness, blur, rotation)
def augment_image(image):
    # Vary brightness
    enhancer = ImageEnhance.Brightness(Image.fromarray(image))
    bright_image = np.array(enhancer.enhance(np.random.uniform(0.5, 1.5)))

    # Apply blur
    blurred_image = cv2.GaussianBlur(bright_image, (5, 5), 0)

    # Generate random rotation (between -10 and 10 degrees)
    angle = np.random.uniform(-10, 10)
    rotated_image = rotate(blurred_image, angle, preserve_range=True).astype(np.uint8)

    return rotated_image

In [ ]:
preprocessed_images = []
for image in images:
    # Preprocess (resize and convert to JPG)
    processed_image = preprocess_image(image)

    # Augment (vary brightness, blur, rotate)
    augmented_image = augment_image(processed_image)

    # Normalize pixel values (optional: normalize to [0, 1] range)
    augmented_image = augmented_image / 255.0

    # Append preprocessed and augmented image to list
    preprocessed_images.append(augmented_image)

# Convert preprocessed images to numpy array
preprocessed_images = np.array(preprocessed_images)

# Print shape of preprocessed images array
print("Shape of preprocessed images:", preprocessed_images.shape)

<ipython-input-6-b77e42aa8009>:7: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  img_bytes = cv2.imencode('.jpg', resized_image)[1].tostring()


Shape of preprocessed images: (857, 128, 128, 3)


In [ ]:
# save preprocessed images list to processed data folder
hdf5_file_path = '/content/drive/MyDrive/IS4242/processed_data/preprocessed_images_h5'

# Create HDF5 file and save the data
with h5py.File(hdf5_file_path, 'w') as hf:
    hf.create_dataset('preprocessed_images', data=preprocessed_images)

print(f"HDF5 file saved successfully at: {hdf5_file_path}")

HDF5 file saved successfully at: /content/drive/MyDrive/IS4242/processed_data/preprocessed_images_h5


In [ ]:
import sys

In [ ]:
sys.getsizeof(preprocessed_images)

336986272